In [ ]:
# default_exp simulators.simulator

# Simulator

> Interface for working with quantum state simulators.

In [ ]:
# hide
from nbdev.showdoc import *

`Simulator` is a base class that all simulators will inherit from. It defines two methods `_apply_gate` and `run`, of which only the latter `run` method exposes the main functionality of a simulator in which we are interested in for applications.

In [ ]:
#export
class Simulator:
    """Interface for quantum state simulation"""
    
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        
    def _apply_gate(self, gate_symbol, qubits):
        """Apply a gate to the `qubits` of the current state."""
        gate = getattr(self, gate_symbol)    
        args = (qubits,) if type(qubits)==int else qubits
        return gate(*args)
    
    def run(self, circuit, fault_circuit=None):
        """Apply gates in `circuit` sequentially to current state.
        If `fault_circuit` is specified apply fault gates at end of each tick."""
        
        msmt_res = []
        for tick_index in range(circuit.n_ticks):
            
            for gate, qubits in circuit[tick_index].items():
                for qubit in qubits:
                    res = self._apply_gate(gate, qubit)
                    if res==0 or res==1: 
                        msmt_res.append( res )
                    
            for f_gate, f_qubits in fault_circuit[tick_index].items():
                for f_qubit in f_qubits:
                    self._apply_gate(f_gate, f_qubit)
                    
        return int(''.join(map(str,msmt_res)), 2)

A quantum state simulation in the circuit model always consists of some representation of a quantum state and some gates (or a sequence of gates, i.e. a `Circuit`) we want to (sequentially) apply to this state. Which gates we can apply depends on the specific simulator we are using. Stabilizer simulators for instance do not include any continuous rotations, while state vector simulators do. Thus, we would expect our `Simulator` object to always yield an error when we try to apply gates to it which the simulator cannot simulate. Let's show this for our base simulator class which doesn't contain any gate definition so far, by making use of the `_apply_gate` method.

In [ ]:
show_doc(Simulator._apply_gate)

<h4 id="Simulator._apply_gate" class="doc_header"><code>Simulator._apply_gate</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>Simulator._apply_gate</code>(**`gate_symbol`**, **`qubits`**)

Apply a gate to the `qubits` of the current state.

In [ ]:
s = Simulator(n_qubits=1)
try:
    s._apply_gate("X", 0) 
except AttributeError as e:
    print(e)

'Simulator' object has no attribute 'X'


As expected, we get an AttributeError, which means the class doesn't implement anything by the name of this gate ("X"). The way we are going to implement gates later in the simulators is by means of functions which may or may not return something (think measurements, which we treat in the same way as "gates" and for which we want to get some result back).

Now, let's focus on the central functionality of the simulator, the `run` function.

In [ ]:
show_doc(Simulator.run)

<h4 id="Simulator.run" class="doc_header"><code>Simulator.run</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>Simulator.run</code>(**`circuit`**, **`fault_circuit`**=*`None`*)

Apply gates in `circuit` sequentially to current state.
If `fault_circuit` is specified apply fault gates at end of each tick.

**TODO** Tests for `run` method